In [1]:
using JuMP
import HiGHS

## Single Knapsack Problem

Reference : https://jump.dev/JuMP.jl/stable/tutorials/linear/knapsack/

### Data

In [2]:
n = 8;
capacity = 102.0;
profit = [15.0, 100.0, 90.0, 60.0, 40.0, 15.0, 10.0, 1.0];
weight = [2.0, 20.0, 20.0, 30.0, 40.0, 30.0, 60.0, 10.0];

### Formulation

In [3]:
model = Model(HiGHS.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: HiGHS

In [4]:
@variable(model, x[1:n], Bin)
@objective(model, Max, sum(profit[i] * x[i] for i in 1:n))
@constraint(model, sum(weight[i] * x[i] for i in 1:n) <= capacity)
print(model)

In [5]:
optimize!(model)
solution_summary(model)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
1 rows, 8 cols, 8 nonzeros
1 rows, 7 cols, 7 nonzeros
Objective function is integral with scale 1

Solving MIP model with:
   1 rows
   7 cols (7 binary, 0 integer, 0 implied int., 0 continuous)
   7 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   331             -inf                 inf        0      0      0         0     0.0s
 S       0       0         0   0.00%   331             265               24.91%        0      0      0         0     0.0s

Solving report
  Status            Optimal
  Primal bound      280
  Dual bound        280
  Gap               0% (tolerance: 0.01%)
  Solution status   feasible
                    280 (objective)
               

* Solver : HiGHS

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "kHighsModelStatusOptimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : NO_SOLUTION
  Objective value    : 2.80000e+02
  Objective bound    : 2.80000e+02
  Relative gap       : 0.00000e+00

* Work counters
  Solve time (sec)   : 1.00231e-03
  Simplex iterations : 2
  Barrier iterations : -1
  Node count         : 1


In [6]:
items_chosen = [i for i in 1:n if value(x[i]) > 0.5]
println("Items chosen: ", items_chosen)

Items chosen: [1, 2, 3, 4, 6]


### Wrting a function

In [7]:
function solve_knapsack_problem(;
    profit::Vector{Float64},
    weight::Vector{Float64},
    capacity::Float64,
)
    n = length(weight)
    # The profit and weight vectors must be of equal length.
    @assert length(profit) == n
    model = Model(HiGHS.Optimizer)
    set_silent(model)
    @variable(model, x[1:n], Bin)
    @objective(model, Max, profit' * x)
    @constraint(model, weight' * x <= capacity)
    #print(model)
    optimize!(model)
    @assert termination_status(model) == OPTIMAL
    @assert primal_status(model) == FEASIBLE_POINT
    
    println("Objective is: ", objective_value(model))
    println("Solution is:")
    for i in 1:n
        print("x[$i] = ", round(Int, value(x[i])))
        println(", c[$i] / w[$i] = ", profit[i] / weight[i])
    end
    chosen_items = [i for i in 1:n if value(x[i]) > 0.5]
    return return chosen_items
end

solve_knapsack_problem(; profit = profit, weight = weight, capacity = capacity)

Objective is: 280.0
Solution is:
x[1] = 1, c[1] / w[1] = 7.5
x[2] = 1, c[2] / w[2] = 5.0
x[3] = 1, c[3] / w[3] = 4.5
x[4] = 1, c[4] / w[4] = 2.0
x[5] = 0, c[5] / w[5] = 1.0
x[6] = 1, c[6] / w[6] = 0.5
x[7] = 0, c[7] / w[7] = 0.16666666666666666
x[8] = 0, c[8] / w[8] = 0.1


5-element Vector{Int64}:
 1
 2
 3
 4
 6

## Multiple Knapsack Problem

<img src="MKP_formulation.jpeg" alt="Math" width="600">

### Data

In [8]:
n = 10;
m = 2;
capacity = [103.0, 156.0];
profit = [78.0, 35.0, 89.0, 36.0, 94.0, 75.0, 74.0, 79.0, 80.0, 16.0];
weight = [18.0, 9.0, 23.0, 20.0, 59.0, 61.0, 70.0, 75.0, 76.0, 30.0];

### Formulation

In [9]:
function solve_multiple_knapsack_problem(;
    profit::Vector{Float64},
    weight::Vector{Float64},
    capacity::Vector{Float64},
)
    n = length(weight)
    m = length(capacity)
    # The profit and weight vectors must be of equal length.
    @assert length(profit) == n
    model = Model(HiGHS.Optimizer)
    set_silent(model) 
    x= @variable(model, x[1:m, 1:n]>=0, Bin)
    @objective(model, Max, sum(profit[j] * x[i, j] for i in 1:m, j in 1:n))
    for i in 1:m
        @constraint(model, weight' * x[i, :] <= capacity[i])
    end
    for j in 1:n
        @constraint(model, sum(x[i, j] for i in 1:m) <= 1)
    end
    #print(model)
    optimize!(model)
    @assert termination_status(model) == OPTIMAL
    @assert primal_status(model) == FEASIBLE_POINT
    print(solution_summary(model))
    return model, x
end

solve_multiple_knapsack_problem (generic function with 1 method)

In [10]:
model_solved, x = solve_multiple_knapsack_problem(; profit = profit, weight = weight, capacity = capacity)

* Solver : HiGHS

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "kHighsModelStatusOptimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : NO_SOLUTION
  Objective value    : 4.52000e+02
  Objective bound    : 4.52000e+02
  Relative gap       : 0.00000e+00

* Work counters
  Solve time (sec)   : 1.01039e-02
  Simplex iterations : 169
  Barrier iterations : -1
  Node count         : 13


(A JuMP Model
Maximization problem with:
Variables: 20
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 12 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 20 constraints
`VariableRef`-in-`MathOptInterface.ZeroOne`: 20 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: HiGHS
Names registered in the model: x, VariableRef[x[1,1] x[1,2] … x[1,9] x[1,10]; x[2,1] x[2,2] … x[2,9] x[2,10]])

### Output

In [11]:
function output_multiple_knapsack_problem(model, x)
    if termination_status(model) == MOI.OPTIMAL || termination_status(model) == MOI.LOCALLY_SOLVED
        println("An optimal solution has been found:")
        println("Optimal objective value: ", objective_value(model))
    end
    println("Solution: ")
    chosen_items = []
    for i in 1:m
        println("::Knapsack $i (Capacity:", capacity[i], ")::")
        for j in 1:n
            print("x[$i, $j] = ", round(Int, value(x[i, j])))
            println(", c[$j] / w[$j] = ", profit[j] / weight[j])
        end
        push!(chosen_items,[j for j in 1:n if value(x[i,j]) > 0.5])
        println("Items chosen: ", chosen_items[i])
        println("Capacity used: ", sum(value(x[i, j]) * weight[j] for j in 1:n))
        println()
    end
    return chosen_items
end

output_multiple_knapsack_problem (generic function with 1 method)

In [12]:
output_multiple_knapsack_problem(model_solved, x)

An optimal solution has been found:
Optimal objective value: 452.0
Solution: 
::Knapsack 1 (Capacity:103.0)::
x[1, 1] = 1, c[1] / w[1] = 4.333333333333333
x[1, 2] = 0, c[2] / w[2] = 3.888888888888889
x[1, 3] = 1, c[3] / w[3] = 3.869565217391304
x[1, 4] = 0, c[4] / w[4] = 1.8
x[1, 5] = 0, c[5] / w[5] = 1.5932203389830508
x[1, 6] = 1, c[6] / w[6] = 1.2295081967213115
x[1, 7] = 0, c[7] / w[7] = 1.0571428571428572
x[1, 8] = 0, c[8] / w[8] = 1.0533333333333332
x[1, 9] = 0, c[9] / w[9] = 1.0526315789473684
x[1, 10] = 0, c[10] / w[10] = 0.5333333333333333
Items chosen: [1, 3, 6]
Capacity used: 102.0

::Knapsack 2 (Capacity:156.0)::
x[2, 1] = 0, c[1] / w[1] = 4.333333333333333
x[2, 2] = 0, c[2] / w[2] = 3.888888888888889
x[2, 3] = 0, c[3] / w[3] = 3.869565217391304
x[2, 4] = 1, c[4] / w[4] = 1.8
x[2, 5] = 1, c[5] / w[5] = 1.5932203389830508
x[2, 6] = 0, c[6] / w[6] = 1.2295081967213115
x[2, 7] = 0, c[7] / w[7] = 1.0571428571428572
x[2, 8] = 0, c[8] / w[8] = 1.0533333333333332
x[2, 9] = 1, c[9]

2-element Vector{Any}:
 [1, 3, 6]
 [4, 5, 9]